#  Overview

This notebook provides step-by-step details on the calculations for Criterion B.

# Setup

Import Python modules

In [1]:
import math
import os

import ee
from gee_redlist.ee_rle import make_eoo, area_km2, get_aoo_grid_projection
import geopandas as gpd
import ipywidgets as widgets
from lonboard import Map, PolygonLayer, BitmapTileLayer

from gee_redlist.ee_auth import initialize_ee
from gee_redlist.ee_rle import load_yaml

Initialize Earth Engine


In [2]:
initialize_ee(project=os.environ['GOOGLE_CLOUD_PROJECT'])

# Analysis

In [3]:
# Define the ecosystem that this notebook is analyzing
ecosystem_code = 'MMR-T1.1.1'

In [4]:
# Load the country config
country_config_path = os.environ['VSCODE_CWD'] + '/config/country_config.yaml'
country_config = load_yaml(country_config_path)
print(f'{country_config = }')

gee_project_path = country_config['gee_project_path']

country_config = {'country_name': 'Myanmar', 'country_code': 'MM', 'gee_project_path': 'projects/goog-rle-assessments/assets', 'classified_image': {'asset_id': 'mm_ecosys_v7b', 'classes': [{'id': 52, 'name': 'Tanintharyi island rainforest', 'code': 'MMR-T1.1.1'}, {'id': 54, 'name': 'Tanintharyi Sundaic lowland evergreen rainforest', 'code': 'MMR-T1.1.2'}, {'id': 53, 'name': 'Tanintharyi limestone tropical evergreen forest', 'code': 'MMR-T1.1.3'}]}}


In [5]:
# Extract the class info for the ecosystem
class_info = [x for x in country_config['classified_image']['classes'] if x['code'] == ecosystem_code][0]
print(f'{class_info = }')

ecosystem_image = {
    'asset_id': f"{gee_project_path}/{ecosystem_code}/{class_info['id']}",
    'pixel_value': class_info['id']
}
print(f'{ecosystem_image = }')

class_info = {'id': 52, 'name': 'Tanintharyi island rainforest', 'code': 'MMR-T1.1.1'}
ecosystem_image = {'asset_id': 'projects/goog-rle-assessments/assets/MMR-T1.1.1/52', 'pixel_value': 52}


In [6]:
classified_image_asset_id = f"{country_config['gee_project_path']}/{country_config['classified_image']['asset_id']}"
print(f'{classified_image_asset_id = }')

class_img = (
    ee.Image(classified_image_asset_id)
      .eq(ecosystem_image['pixel_value'])
      .selfMask()
)
print(f'class_img: {class_img.getInfo()}')

classified_image_asset_id = 'projects/goog-rle-assessments/assets/mm_ecosys_v7b'
class_img: {'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [14299, 23576], 'crs': 'EPSG:4326', 'crs_transform': [0.0008084837557075694, 0, 89.60991403135986, 0, -0.0008084837557075694, 28.548369897789982]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[94.66768735253133, 28.548775583027798], [93.22272474215634, 28.548775580809586], [91.41652148658464, 28.548775539452144], [89.60939625454633, 28.54877430628462], [89.60949715757825, 9.485667642440385], [91.41652148658464, 9.486595817478428], [92.86148408075269, 9.486595814990817], [94.30644671824773, 9.486595784089035], [95.39016867380931, 9.48715124036664], [96.47389058147114, 9.486595770655939], [97.91885317497325, 9.486595836813008], [99.36381579356966, 9.486595799973498], [101.1708401374696, 9.48566765739743], [101.17094098108427, 28.548774348

In [7]:
# Commented out until bug is fixed: https://github.com/developmentseed/lonboard/issues/1064
# # Determine the coordinates for viewing the image
# longitude, latitude = class_img.geometry().centroid().getInfo()['coordinates']
# longitude, latitude

# Extent of occurrence (EOO) (subcriterion B1)

Set the scale (in meters) for reducing the image pixels to polygons. Use the image's nominal scale unless is is less than 50 meters per pixel.

In [8]:
reduction_scale = max(class_img.projection().nominalScale().getInfo(), 50)
reduction_scale

90

Covert the classified image to vectors.

In [9]:
ecosystem_polygons = class_img.updateMask(1).reduceToVectors(
    scale=reduction_scale,
    geometry=class_img.geometry(),
    geometryType='polygon',
    maxPixels=1e12,
    bestEffort=False
)

Create a convex hull that encompasses the ecosystem polygons.

In [10]:
# convexHull() is called twice as a workaround for a bug
# (https://issuetracker.google.com/issues/465490917)
hull = ecosystem_polygons.geometry().convexHull(maxError=1).convexHull(maxError=1)

In [11]:
aoo_area_km2 = hull.area().multiply(1e-6).getInfo()
print(f'The area of the hull is {aoo_area_km2:.0f} km²')

The area of the hull is 50337 km²


## Display EOO Layers

## Ecosystem Tiles

In [35]:
tile_url = class_img.getMapId(
    vis_params={
        'palette': ['blue'],
        'opacity': 0.5
    }
)['tile_fetcher'].url_format

tile_layer = BitmapTileLayer(
    data=tile_url,
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)

## Ecosystem Polygons Layer

In [36]:
ecosystem_polygons_gdf = gpd.GeoDataFrame.from_features(
    ecosystem_polygons.getInfo()['features'],
    crs='EPSG:4326'
)

ecosystem_polygons_layer = PolygonLayer.from_geopandas(
    ecosystem_polygons_gdf,
    get_fill_color=[255, 0, 0, 127],
    stroked=True,
    get_line_width=2,
    get_line_color=[0, 0, 0, 150],
)

## Ecosystem Hull Layer

In [37]:
hull_gdf = gpd.GeoDataFrame.from_features(
    ee.FeatureCollection(hull).getInfo(),
    crs='EPSG:4326'
)

hull_layer = PolygonLayer.from_geopandas(
    hull_gdf,
    get_fill_color=[0, 0, 255, 63],
    stroked=True,
    get_line_width=200,
    get_line_color=[0, 0, 0, 150],
)

Display the map.

In [39]:
m = Map(
    layers=[
        ecosystem_polygons_layer,
        tile_layer,
        hull_layer
    ],
    controls=[]
)

checkbox_hull = widgets.Checkbox(value=True, description='Hull')
checkbox_tiles = widgets.Checkbox(value=False, description='Tiles')
checkbox_ecosystem_polygons = widgets.Checkbox(value=True, description='Gridcells')

# Link checkboxes to layer visibility
widgets.link((checkbox_hull, 'value'), (hull_layer, 'visible'))
widgets.link((checkbox_tiles, 'value'), (tile_layer, 'visible'))
widgets.link((checkbox_ecosystem_polygons, 'value'), (ecosystem_polygons_layer, 'visible'))

controls = widgets.VBox([checkbox_tiles, checkbox_ecosystem_polygons, checkbox_hull])
display(widgets.HBox([controls, m]))

## Verify that the step-by-step results are consistent

In [15]:
# Direct call to `make_eoo()`
aoo_area_km2_direct_call = area_km2(make_eoo(class_img)).getInfo()
print(f'EOO area: {aoo_area_km2_direct_call:.0f} km²')

EOO area: 50337 km²


In [16]:
# Assert the two values are close to each other.
assert math.isclose(aoo_area_km2, aoo_area_km2_direct_call, abs_tol=1e-4)

# Area of Occupancy (AOO) (subcriterion B2)

The protocol for this adjustment includes the following steps:

> - Intersect AOO grid with the ecosystem’s distribution map.
> - Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).
> - Arrange grid cells in ascending order based on their area (smaller first).
> - Calculate accumulated sum of area per cell (‘cumulative area’).
> - Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).
> - Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

## Intersect AOO grid with the ecosystem’s distribution map

Load the AOO grid projection

In [17]:
aoo_grid_proj = get_aoo_grid_projection()

aoo_grid_proj.getInfo()

{'type': 'Projection',
 'wkt': 'PROJCS["World_Cylindrical_Equal_Area", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Cylindrical_Equal_Area"], \n  PARAMETER["central_meridian", 0.0], \n  PARAMETER["standard_parallel_1", 0.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH], \n  AUTHORITY["ESRI","54034"]]',
 'transform': [10000, 0, 0, 0, 10000, 0]}

Extract the grid scale parameters

In [18]:
aoo_x_scale, _, _, _, aoo_y_scale, _ = aoo_grid_proj.getInfo()['transform']
print(f'{aoo_x_scale = } meters')
print(f'{aoo_y_scale = } meters')

aoo_x_scale = 10000 meters
aoo_y_scale = 10000 meters


> - Create an Earth Engine feature collection of AOO grid cells that intersect with the ecosystem, and calculate the fractional coverage of the ecosystem within the grid cell.

In [19]:
fractional_coverage_fc = class_img.unmask().reduceRegions(
  collection=class_img.geometry().coveringGrid(aoo_grid_proj),
  reducer=ee.Reducer.mean(),
).filter(ee.Filter.gt('mean', 0))

# Convert the Earth Engine feature collection to a GeoPandas GeoDataFrame.
fractional_coverage_gdf = ee.data.computeFeatures({
    "expression": fractional_coverage_fc,
    "fileFormat": "GEOPANDAS_GEODATAFRAME",
})
fractional_coverage_gdf.rename(columns={"mean": "coverage"}, inplace=True)

In [20]:
len(fractional_coverage_gdf)

206

## Calculate grid cell area and the total ecosystem area

> - Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).

In [21]:
fractional_coverage_gdf['area'] = fractional_coverage_gdf['coverage'] * aoo_x_scale * aoo_y_scale

fractional_coverage_gdf.sort_values(by="area")[0:4]

,geometry,coverage,area
175,"POLYGON ((98.09603 12.57845, 98.18586 12.57845...",0.000035,3481.367444
125,"POLYGON ((98.09603 11.93102, 98.18586 11.93102...",0.000079,7875.324046
111,"POLYGON ((98.09603 11.74632, 98.18586 11.74632...",0.000079,7880.464175
74,"POLYGON ((98.54519 11.1008, 98.63502 11.1008, ...",0.000079,7898.091697


In [22]:
total_area_km2 = fractional_coverage_gdf['area'].sum() / 1e6
print(f'Total ecosystem area: {total_area_km2:.0f} km²')

Total ecosystem area: 1937 km²


## Calculate cumulative area in ordered cells

> - Arrange grid cells in ascending order based on their area (smaller first).
> - Calculate accumulated sum of area per cell (‘cumulative area’).

In [23]:
fractional_coverage_gdf = fractional_coverage_gdf.sort_values(by="area")
fractional_coverage_gdf["cumulative_area"] = fractional_coverage_gdf["area"].cumsum()

fractional_coverage_gdf.sort_values(by="area").head()

,geometry,coverage,area,cumulative_area
175,"POLYGON ((98.09603 12.57845, 98.18586 12.57845...",0.000035,3481.367444,3481.367444
125,"POLYGON ((98.09603 11.93102, 98.18586 11.93102...",0.000079,7875.324046,11356.691491
111,"POLYGON ((98.09603 11.74632, 98.18586 11.74632...",0.000079,7880.464175,19237.155665
74,"POLYGON ((98.54519 11.1008, 98.63502 11.1008, ...",0.000079,7898.091697,27135.247363
160,"POLYGON ((97.91637 12.39331, 98.0062 12.39331,...",0.000123,12301.457861,39436.705224


## Calculate the cumulative proportion

> - Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).

In [24]:
fractional_coverage_gdf["cumulative_proportion"] = fractional_coverage_gdf["cumulative_area"] / 1e6 / total_area_km2

fractional_coverage_gdf

,geometry,coverage,area,cumulative_area,cumulative_proportion
175,"POLYGON ((98.09603 12.57845, 98.18586 12.57845...",0.000035,3.481367e+03,3.481367e+03,0.000002
125,"POLYGON ((98.09603 11.93102, 98.18586 11.93102...",0.000079,7.875324e+03,1.135669e+04,0.000006
111,"POLYGON ((98.09603 11.74632, 98.18586 11.74632...",0.000079,7.880464e+03,1.923716e+04,0.000010
74,"POLYGON ((98.54519 11.1008, 98.63502 11.1008, ...",0.000079,7.898092e+03,2.713525e+04,0.000014
160,"POLYGON ((97.91637 12.39331, 98.0062 12.39331,...",0.000123,1.230146e+04,3.943671e+04,0.000020
...,...,...,...,...,...
108,"POLYGON ((98.45536 11.65401, 98.54519 11.65401...",0.549146,5.491457e+07,1.689946e+09,0.872610
102,"POLYGON ((98.45536 11.56174, 98.54519 11.56174...",0.561263,5.612625e+07,1.746072e+09,0.901591
164,"POLYGON ((98.36552 12.39331, 98.45536 12.39331...",0.586257,5.862569e+07,1.804698e+09,0.931863
99,"POLYGON ((98.18586 11.56174, 98.27569 11.56174...",0.607082,6.070816e+07,1.865406e+09,0.963210


## Calculate AOO

> - Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

In [25]:
aoo_grid_cells = fractional_coverage_gdf[fractional_coverage_gdf["cumulative_proportion"] > 0.01]
print(f"AOO (number of cells with cumulative proportion > 0.01): {len(aoo_grid_cells)}")

AOO (number of cells with cumulative proportion > 0.01): 139


In [26]:
aoo_grid_cells_dropped = fractional_coverage_gdf[fractional_coverage_gdf["cumulative_proportion"] <= 0.01]

## Display the layers

In [27]:
aoo_grid_cells_layer = PolygonLayer.from_geopandas(
    aoo_grid_cells,
    get_fill_color=[0, 255, 0, 63],
)

aoo_grid_cells_layer_dropped = PolygonLayer.from_geopandas(
    aoo_grid_cells_dropped,
    get_fill_color=[255, 0, 0, 63],
)

m = Map(
    layers=[
        tile_layer,
        aoo_grid_cells_layer,
        aoo_grid_cells_layer_dropped
    ],
    controls=[]
)

controls = widgets.HBox([])
display(controls, m)

/Users/tylere/Documents/GitHub/VorGeo/rle-assessment-report/.pixi/envs/default/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:40: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/Users/tylere/Documents/GitHub/VorGeo/rle-assessment-report/.pixi/envs/default/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:40: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


HBox()

## Verify that the step-by-step results are consistent

In [ ]:
# Direct call to `make_aoo()`
aoo_cells_direct_call = make_aoo(class_img)

print(f'AOO: {aoo_grid_cell_count_direct_call} grid cells')

In [31]:
# assert aoo_grid_cell_count == aoo_grid_cell_count_direct_call

# Criterion B Summary

In [32]:
print(f'As shown in the preceding sections, '
      f'AOO and EOO were measured as '
      f'{aoo_grid_cell_count} 10 x 10 km grid cells '
      f'and {aoo_area_km2:.0f} km², respectively.')

NameError: name 'aoo_grid_cell_count' is not defined